In [1]:
import pandas as pd
# 판다스 라이브러리 불러오기

df = pd.read_excel('기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20240101).xlsx')
# 엑셀 데이터 불러오기

sj = df[df['2단계']=='성남시중원구']
sj.head()

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
1345,kor,4113300000,경기도,성남시중원구,NaN,63,124,127,8,21.91,37,25,39.61,127.139419,37.427669,NaN
1346,kor,4113351000,경기도,성남시중원구,성남동,63,124,127,8,0.71,37,25,44.67,127.133531,37.429075,NaN
1347,kor,4113352500,경기도,성남시중원구,중앙동,63,124,127,9,26.90,37,26,38.31,127.157472,37.443975,20140319.0
1348,kor,4113353000,경기도,성남시중원구,금광1동,63,124,127,9,52.32,37,26,32.05,127.164533,37.442236,NaN
1349,kor,4113354000,경기도,성남시중원구,금광2동,63,124,127,9,58.08,37,26,36.50,127.166133,37.443472,NaN


# 공공데이터를 활용하기

In [2]:
import requests
import json

# 발급받은 API 키를 입력합니다.
api_key = "UhIXqz3BxyzUBIOobK+yC+lbhlKx4lOWT5glOai8uSuRyW/ea2D6OBF7zp5WnyjTYcXyrzk+PXV2KtNrChOzOQ=="


# 기상청 날씨 데이터를 요청할 URL을 구성합니다.
# 이 URL은 예시일 뿐, 실제 URL은 공공데이터 포털에서 제공하는 API 문서를 참고하세요.
url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"


def get_weather(nx,ny):
    
    params = {
        'serviceKey': api_key,
        'numOfRows': '10',
        'pageNo': '1',
        'dataType': 'JSON',
        'base_date': '20240611',  # 기준 날짜 (YYYYMMDD)
        'base_time': '0500',      # 기준 시간 (HHMM)
        'nx': nx,               # 격자 X 좌표
        'ny': ny               # 격자 Y 좌표
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data != None:
            items = data['response']['body']['items']['item']
        
            # 데이터를 출력합니다.
            for item in items:
                print(f"예보일자: {item['fcstDate']}, 예보시간: {item['fcstTime']}, 예보값: {item['fcstValue']}, 카테고리: {item['category']}")
        else:
            return print(f"Error: {response.status_code}")
    
    else:
        print(f"Error: {response.status_code}")

In [3]:
def get_lat_lon(address):
    # Nominatim API의 엔드포인트 URL을 구성합니다.
    url = "https://nominatim.openstreetmap.org/search"
    
    # 요청할 파라미터를 설정합니다.
    params = {
        'q': address,
        'format': 'json'
    }
    
    # API 요청을 보냅니다.
    response = requests.get(url, params=params)
    
    # 응답이 성공적인지 확인합니다.
    if response.status_code == 200:
        data = response.json()
        if data:
            # 첫 번째 결과를 사용합니다.
            latitude = data[0].get('lat', 'N/A')
            longitude = data[0].get('lon', 'N/A')
            return latitude, longitude
        else:
            return "N/A", "N/A"
    else:
        return f"Error: {response.status_code}"


# 예시 주소를 입력합니다.
address = "경기도 성남시 분당구 성남대로 601"

# 위도와 경도를 가져옵니다.
latitude, longitude = get_lat_lon(address)
print(f"Latitude: {latitude}, Longitude: {longitude}")

latitude= float(latitude)
longitude = float(longitude)

print(type(latitude))
get_weather(int(latitude),int(longitude))


Latitude: 37.4113471, Longitude: 127.1288223
<class 'float'>
예보일자: 20240611, 예보시간: 0600, 예보값: 19, 카테고리: TMP
예보일자: 20240611, 예보시간: 0600, 예보값: 1.3, 카테고리: UUU
예보일자: 20240611, 예보시간: 0600, 예보값: 1.1, 카테고리: VVV
예보일자: 20240611, 예보시간: 0600, 예보값: 230, 카테고리: VEC
예보일자: 20240611, 예보시간: 0600, 예보값: 1.7, 카테고리: WSD
예보일자: 20240611, 예보시간: 0600, 예보값: 1, 카테고리: SKY
예보일자: 20240611, 예보시간: 0600, 예보값: 0, 카테고리: PTY
예보일자: 20240611, 예보시간: 0600, 예보값: 0, 카테고리: POP
예보일자: 20240611, 예보시간: 0600, 예보값: 0.5, 카테고리: WAV
예보일자: 20240611, 예보시간: 0600, 예보값: 강수없음, 카테고리: PCP
